In [25]:
fin = open("track1/pair_train.txt", "r")
m = countlines(fin)

11765848

In [2]:
# build item_dict
f_item = open("track1/item.txt", "r")
n = countlines(f_item)
seekstart(f_item)
item_dict = Dict{Int32, Int32}()
for i = 1:n
    x = split(readline(f_item), '\t')
    item_dict[parse(Int32, x[1])] = i
end

In [3]:
struct User
    id::Int32
    age::Int16
    gender::Int8
    cnt_tweet::Int32
    tag_list::Array{Int32, 1}
end

In [4]:
f_user_profile = open("track1/user_profile.txt", "r")
cnt_user = countlines(f_user_profile)

Tag_dict = Dict{Int32, Int32}()
id_tag = 0
seekstart(f_user_profile)
U = Array{User, 1}()
for i = 1:cnt_user
    x = split(readline(f_user_profile), '\t')
    tag_list = map(t->parse(Int32, t), split(x[5], ';'))
    (tag_list[1] == 0) && (tag_list = Array{Int32, 1}())
    for id in tag_list
        (haskey(Tag_dict, id)) && continue
        id_tag += 1
        Tag_dict[id] = id_tag
    end
    try
        x = map(t->parse(Int32, t), x[1:4])
        x[2] = (x[2] < 1950) ? 0 :
               (1950 ≤ x[2] < 2004) ? ceil((x[2] - 1950) / 2) + 1 : 
               (x[2] ≥ 2004) ? 28 : 29
    catch ArgumentError
        x[2] = "29"
        x = map(t->parse(Int32, t), x[1:4])
    end
    x[4] = x[4] != 0 ? floor(log2(x[4])) : 0
    x[2] += 1
    x[3] += 1
    x[4] += 1
    push!(U, User(x[1], x[2], x[3], x[4], tag_list))
end
println(id_tag)

130304


In [5]:
# build user_dict
user_dict = Dict{Int32, Int32}()
id = 0
for u in U
    id += 1
    user_dict[u.id] = id
end

In [6]:
f_user_sns = open("track1/user_sns.txt", "r")
len_sns = countlines(f_user_sns)

50655143

In [7]:
seekstart(f_user_sns)
S = Array{Array{Int32, 1}, 1}()
for i = 1:cnt_user
    push!(S, Array{Int32, 1}())
end
for i = 1:len_sns
    x = map(t->user_dict[parse(Int32, t)], split(readline(f_user_sns), '\t'))
    push!(S[x[1]], x[2])
end

In [8]:
f_user_action = open("track1/user_action.txt", "r")
len_action = countlines(f_user_action)

10632833

In [9]:
seekstart(f_user_action)
A = Array{Array{Int32, 1}, 1}()
for i = 1:cnt_user
    push!(A, Array{Int32, 1}())
end
for i = 1:len_action
    x = map(t->parse(Int32, t), split(readline(f_user_action), '\t'))
    push!(A[user_dict[x[1]]], user_dict[x[2]])
end

In [10]:
f_item = open("track1/item.txt", "r")
m_item = countlines(f_item)

6095

In [11]:
Item_key = Dict{Int32, Array{Int32, 1}}()
Key_dict = Dict{Int32, Int32}()
key_id = 0
seekstart(f_item)
for i = 1:m_item
    x = split(readline(f_item), '\t')
    item_id = item_dict[parse(Int32, x[1])]
    Item_key[item_id] = map(t->parse(Int32, t), split(x[3], ';'))
    for x in Item_key[item_id]
        if !haskey(Key_dict, x)
            key_id += 1
            Key_dict[x] = key_id
        end
    end
end
println(key_id)

20935


In [12]:
f_user_keyword = open("track1/user_key_word.txt", "r")
m_user_keyword = countlines(f_user_keyword)

2320895

In [13]:
User_key = Dict{Int32, Array{Tuple{Int32, Float32}, 1}}()
seekstart(f_user_keyword)
for i = 1:m_user_keyword
    line = split(readline(f_user_keyword))
    user_id = user_dict[parse(Int32, line[1])]
    keywords = map(t->split(t, ':'), split(line[2], ';'))
    keywords = map((t->(parse(Int32, t[1]), parse(Float32, t[2]))), keywords)
    User_key[user_id] = keywords
    for x in User_key[user_id]
        if !haskey(Key_dict, x[1])
            key_id += 1
            Key_dict[x[1]] = key_id
        end
    end
end
println(key_id)

255290


In [26]:
seekstart(fin)
#m = 1000000
Ω = Array{Any, 1}()
for i = 1:m
    x = map(t->parse(Int32, t), split(readline(fin), ' '))
    x[1], x[3], x[5] = user_dict[x[1]], item_dict[x[3]], item_dict[x[5]]
    push!(Ω, x)
end

In [ ]:
function sigmoid(x)
    return 1 / (1 + exp(-x))
end

function cal_timeb(t)
    bₜ = (t - d⁻) / (d⁺ - d⁻) * bₜ⁻ + (d⁺ - t) / (d⁺ - d⁻) * bₜ⁺
    s = (t - d⁻) % 86400
    bₛ = (s - s⁻) / (s⁺ - s⁻) * bₛ⁻ + (s⁺ - s) / (s⁺ - s⁻) * bₛ⁺
    hour = s ÷ 3600
    return bₕ[hour + 1] + bₜ + bₛ
end

function cal_timeq(t)
    zₜ = (t - d⁻) / (d⁺ - d⁻) * zₜ⁻ + (d⁺ - t) / (d⁺ - d⁻) * zₜ⁺
    s = (t - d⁻) % 86400
    zₛ = (s - s⁻) / (s⁺ - s⁻) * zₛ⁻ + (s⁺ - s) / (s⁺ - s⁻) * zₛ⁺
    return zₜ + zₛ
end

function update_timeb(t, Δ)
    global bₜ⁻ += (t - d⁻) / (d⁺ - d⁻) * Δ
    global bₜ⁺ += (d⁺ - t) / (d⁺ - d⁻) * Δ
    s = (t - d⁻) % 86400
    global bₛ⁻ += (s - s⁻) / (s⁺ - s⁻) * Δ
    global bₛ⁺ += (s⁺ - s) / (s⁺ - s⁻) * Δ
end

function update_timeq(t, Δ)
    global zₜ⁻ += (t - d⁻) / (d⁺ - d⁻) * Δ
    global zₜ⁺ += (d⁺ - t) / (d⁺ - d⁻) * Δ
    s = (t - d⁻) % 86400
    global zₛ⁻ += (s - s⁻) / (s⁺ - s⁻) * Δ
    global zₛ⁺ += (s⁺ - s) / (s⁺ - s⁻) * Δ
end


d = 60
α = 0.001
d⁻, d⁺ = 1318348785, 1322668798
s⁻, s⁺ = 0, 86400
#=
iter = 0
bₕ = rand(1, 24)
bₖ = rand(1, key_id)
bₜ⁻, bₜ⁺ = rand(), rand()
bₛ⁻, bₛ⁺ = rand(), rand()
zₜ⁻, zₜ⁺ = rand(d, 1), rand(d, 1)
zₛ⁻, zₛ⁺ = rand(d, 1), rand(d, 1)
p = rand(d, cnt_user)
y = rand(d, cnt_user)
q = rand(d, n)
bᵢ = rand(1, n)
bₐ = rand(30, n)
bᵧ = rand(4, n)
yₐ = rand(d, 30, 4)
yₜ = rand(d, 16)
yₖ = rand(d, key_id)
yₚ = rand(d, id_tag)
=#
#Ω = Ω[1:1000000]
#m = 1000000


lst_error = 0

while true
    try
        iter += 1
        error = 0
        for x in Ω
            u = U[x[1]]
            user_key = Dict{Int32, Float32}()
            for key in User_key[x[1]]
                user_key[key[1]] = key[2]
            end
            # Calculate pᵤ
            p_tot = p[:, x[1]]
            for xx in S[x[1]]
                p_tot += y[:, xx] / (size(S[x[1]], 1)^0.4)
            end
            for xx in A[x[1]]
                p_tot += y[:, xx] / (size(A[x[1]], 1)^0.5)
            end
            p_tot += yₐ[:, u.age, u.gender] + yₜ[:, u.cnt_tweet]
            for key in User_key[x[1]]
                p_tot += key[2] * yₖ[:, Key_dict[key[1]]]
            end
            for tag in u.tag_list
                p_tot += yₚ[:, Tag_dict[tag]] / (size(u.tag_list, 1)^0.5)
            end
            # Calculate bᵢ
            b = bᵢ[x[5]] - bᵢ[x[3]] 
                + bₐ[u.age, x[5]] - bₐ[u.age, x[3]] 
                + bᵧ[u.gender, x[5]] - bᵧ[u.gender, x[3]]
                + cal_timeb(x[4]) - cal_timeb(x[2])
            for key in Item_key[x[5]]
                (!haskey(user_key, key)) && continue
                b += bₖ[Key_dict[key]]
            end
            for key in Item_key[x[3]]
                (!haskey(user_key, key)) && continue
                b -= bₖ[Key_dict[key]]
            end
            # Calculate qᵢ
            qᵢ = q[:, x[5]] - q[:, x[3]] + cal_timeq(x[4]) - cal_timeq(x[2])
            
            # Predict Error
            r = sigmoid(b + qᵢ[:]' * p_tot)
            error += r
            
            # Update bᵢ
            bᵢ[x[5]] += α * r * (1 - r)
            bₐ[u.age, x[5]] += α * r * (1 - r)
            bᵧ[u.gender, x[5]] += α * r * (1 - r)
            update_timeb(x[4], α * r * (1 - r))
            bᵢ[x[3]] -= α * r * (1 - r)
            bₐ[u.age, x[3]] -= α * r * (1 - r)
            bᵧ[u.gender, x[3]] -= α * r * (1 - r)
            update_timeb(x[2], - α * r * (1 - r))
            for key in Item_key[x[5]]
                (!haskey(user_key, key)) && continue
                bₖ[Key_dict[key]] += α * r * (1 - r)
            end
            for key in Item_key[x[3]]
                (!haskey(user_key, key)) && continue
                bₖ[Key_dict[key]] -= α * r * (1 - r)
            end
            # Update pᵤ
            for xx in S[x[1]]
                y[:, xx] += (α * r * (1 - r) / (size(S[x[1]], 1)^0.4)) * qᵢ
            end
            for xx in A[x[1]]
                y[:, xx] += (α * r * (1 - r) / (size(A[x[1]], 1)^0.5)) * qᵢ
            end
            p[:, x[1]] += α * r * (1 - r) * qᵢ
            yₐ[:, u.age, u.gender] += α * r * (1 - r) * qᵢ
            yₜ[:, u.cnt_tweet] += α * r * (1 - r) * qᵢ
            for key in User_key[x[1]]
                yₖ[:, Key_dict[key[1]]] += key[2] * α * r * (1 - r) * qᵢ
            end
            for tag in u.tag_list
                yₚ[:, Tag_dict[tag]] += α * r * (1 - r) / (size(u.tag_list, 1)^0.5) * qᵢ
            end
            # Update qᵢ
            q[:, x[5]] += α * r * (1 - r) * p_tot
            update_timeq(x[4], α * r * (1 - r) * p_tot)
            q[:, x[3]] -= α * r * (1 - r) * p_tot
            update_timeq(x[2], - α * r * (1 - r) * p_tot)
        end
        error /= m
        (iter % 1 == 0) && println("$iter $error")
        lst_error = error
    catch InterruptException
        break
    end
end
println(lst_error)

101 0
102 0.7464357349112024
103 0.7488766174344257


In [24]:
function read_from_log_test(f)
    x = split(readline(f), '\t')
    x = map(t->parse(Int32, t), x[1:4])
    return x
end

function test(f_rec_log_test, fout, key)
    m_test = countlines(f_rec_log_test)
    seekstart(f_rec_log_test)
    user_cnt, i = 0, 1
    cur = read_from_log_test(f_rec_log_test)
    # m_test = 800
    while i <= m_test
        j = i + 1
        user_cnt += 1
        _Ω = Array{Any, 1}()
        push!(_Ω, cur)
        while j <= m_test
            cur = read_from_log_test(f_rec_log_test)
            (cur[1] != _Ω[1][1]) && break
            j += 1
            push!(_Ω, cur)
        end
        (i ÷ 5000000 != j ÷ 5000000) && println(i, ' ', j)
        dict = Array{Any, 1}()
        x₁ = user_dict[_Ω[1][1]]
        u = U[x₁]
        user_key = Dict{Int32, Float32}()
        for key in User_key[x₁]
            user_key[key[1]] = key[2]
        end
        p_tot = p[:, x₁]
        for xx in S[x₁]
            p_tot += y[:, xx[1]] / (size(S[x₁], 1)^0.4)
        end
        for xx in A[x₁]
            p_tot += y[:, xx[1]] / (size(A[x₁], 1)^0.5)
        end
        p_tot += yₐ[:, u.age, u.gender] + yₜ[:, u.cnt_tweet]
        for key in User_key[x₁]
            p_tot += key[2] * yₖ[:, Key_dict[key[1]]]
        end
        for tag in u.tag_list
            p_tot += yₚ[:, Tag_dict[tag]] / (size(u.tag_list, 1)^0.5)
        end
        for x in _Ω[1:j-i]
            x₂ = item_dict[x[2]]
            b = bᵢ[x₂] + cal_timeb(x[4]) + bₐ[u.age, x₂] + bᵧ[u.gender, x₂]
            for key in Item_key[x₂]
                (!haskey(user_key, key)) && continue
                b += bₖ[Key_dict[key]]
            end
            qᵢ = q[:, x₂] + cal_timeq(x[4])
            score = b + qᵢ[:]' * p_tot
            push!(dict, (score, x[2]))
        end
        sort!(dict)
        write(fout, "$(_Ω[1][1]),")
        for k = 1:min(size(dict, 1), 3)
            t = size(dict, 1) - k + 1
            write(fout, ((k == 1) ? "" : " ") * "$(dict[t][2])")
        end
        write(fout, "\n")
        i = j
    end
end

fout = open("test.csv", "w")
write(fout, "id,clicks\n")

# test(open("track1/rec_log_test_sorted.txt", "r"), fout, "Public")
test(open("track1/rec_log_test_pub.txt", "r"), fout, "Public")
test(open("track1/rec_log_test_pri.txt", "r"), fout, "Private")

close(fout)

4999977 5000050
9999990 10000050
14999999 15000020
4999988 5000005
9999954 10000009
14999996 15000019
